# 1. Imports & Config

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy import ndarray
from matplotlib.pyplot import subplots, show
from sklearn.model_selection import train_test_split

# model building imports
import keras
from keras import Model, Sequential, Input
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.layers import LeakyReLU
from keras.ops import add
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy, AUC, F1Score
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler ,ReduceLROnPlateau, EarlyStopping
from keras.utils import to_categorical

import keras_tuner
# augmentation operations
from keras.layers import RandomBrightness, RandomFlip, RandomRotation
from keras.layers import Pipeline
from keras.layers import BatchNormalization
from keras.layers import Dropout

In [2]:
import tensorflow as tf

# List all physical GPUs
gpus = tf.config.list_physical_devices('GPU')
print("Available GPUs:", gpus)


Available GPUs: []


In [3]:
print('num gpus:', len(tf.config.list_physical_devices('GPU')))

num gpus: 0


In [6]:
# importing the dataset
metadata = pd.read_csv('../Data/rare_species/metadata.csv')
metadata.sample(5)

,rare_species_id,eol_content_id,eol_page_id,kingdom,phylum,family,file_path
6229,7449a525-55e0-46d2-8023-37869ad554ca,20206310,46561068,animalia,chordata,urolophidae,chordata_urolophidae/20206310_46561068_eol-ful...
3968,d8307832-301f-48fa-b962-90b8943bff65,22517379,46578204,animalia,chordata,chaetodontidae,chordata_chaetodontidae/22517379_46578204_eol-...
3952,656f2355-1d18-4e6f-9040-fa241e5a28e8,20312745,323914,animalia,chordata,atelidae,chordata_atelidae/20312745_323914_eol-full-siz...
8888,2493e2df-3c2c-4af1-9afd-510635e314c0,29574828,45511296,animalia,chordata,procellariidae,chordata_procellariidae/29574828_45511296_eol-...
560,03be01cb-b50f-483a-a736-91ffcb78a0f8,28911621,7250886,animalia,chordata,indriidae,chordata_indriidae/28911621_7250886_eol-full-s...


In [7]:
# applying the filepath
metadata['file_path'] = metadata['file_path'].apply(lambda x: os.path.join('../Data/rare_species', x))

In [8]:
metadata.sample(5)

,rare_species_id,eol_content_id,eol_page_id,kingdom,phylum,family,file_path
1547,e81271c9-3ef5-4d99-bf26-06d67accb98a,29468942,795274,animalia,chordata,crocodylidae,../Data/rare_species\chordata_crocodylidae/294...
10231,fb576395-add5-464d-97f6-7bebabb21cba,28221223,45276460,animalia,cnidaria,agariciidae,../Data/rare_species\cnidaria_agariciidae/2822...
79,b380aba7-71ef-4931-93db-a9b8d2676bb6,20944394,1065125,animalia,chordata,anatidae,../Data/rare_species\chordata_anatidae/2094439...
8698,c0225de8-3a63-4a54-9eae-4fc927c62219,29467981,46579618,animalia,chordata,serranidae,../Data/rare_species\chordata_serranidae/29467...
2363,4f08814d-ecff-4629-ba8e-7e542512dafa,29856165,485420,animalia,arthropoda,formicidae,../Data/rare_species\arthropoda_formicidae/298...


# 2.  Preprocessing

In [9]:
metadata.phylum.unique()

array(['mollusca', 'chordata', 'arthropoda', 'echinodermata', 'cnidaria'],
      dtype=object)

In [10]:
print(metadata['family'].value_counts())

family
dactyloidae        300
cercopithecidae    300
formicidae         291
carcharhinidae     270
salamandridae      270
                  ... 
cyprinodontidae     30
alligatoridae       30
balaenidae          30
goodeidae           30
siluridae           29
Name: count, Length: 202, dtype: int64


In [15]:
num_classes = metadata['family'].nunique()
num_classes

202

our dataset is inbalaced and has a length of 202 classes

In [11]:
# Transforming our target and feature into a int
metadata['target'] = metadata['family'].astype('category').cat.codes
metadata['feat'] = metadata['phylum'].astype('category').cat.codes

In [14]:
metadata['target'].sample(10)

7895      75
1435       4
6346      60
3627     122
11824     66
8034     104
11592      1
6971       3
3068     167
3516      10
Name: target, dtype: int16

### Spliting the data

In [19]:
# SSpliting the data while keeping the same distribuition of classes(target) - 70%/20%/10%

train, temp = train_test_split(metadata, test_size=0.3,stratify=metadata['target'],random_state=1)

val, test = train_test_split(temp,test_size=1/3,stratify=temp['target'],random_state=1)

In [20]:
train.shape , val.shape , test.shape

((8388, 9), (2396, 9), (1199, 9))

Loading the images

In [ ]:
def load_image(df, path='file_path', label='target', image_size=(256, 256)):
    X = []
    y = []

    for _, row in df.iterrows():
        image = keras.utils.load_img(row[path], target_size=image_size)
        input_arr = keras.utils.img_to_array(image) / 255.0  # Normalize
        X.append(input_arr)
        y.append(row[label])
    
    return np.array(X), np.array(y)

In [ ]:
X_train , y_train = load_image(train)
X_val , y_val = load_image(val)
X_test , y_test = load_image(test)

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

In [ ]:
y_train = to_categorical(y_train, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

# 3. Models

In [ ]:
augmentation_layer = Sequential(
    [
        RandomBrightness(factor=0.2, value_range=(0.0, 1.0)),
        RandomFlip(),
        RandomRotation(factor=0.2, fill_mode="reflect")
    ], 
    name="augmentation_layer")


In [ ]:
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0, verbose = 1)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20, restore_best_weights = True)

In [ ]:
optimizer = SGD(learning_rate=0.01, name="optimizer")
loss = CategoricalCrossentropy(name="loss")

# metrics
categorical_accuracy = CategoricalAccuracy(name="accuracy")
f1_score = F1Score(average="macro", name="f1_score")
metrics = [categorical_accuracy]

In [ ]:
input_shape = (256,256,3)

epochs = 200
batch_size = 32

## Model 1

In [ ]:
def build_transfg_like(input_shape=input_shape, num_classes=num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), activation="relu", padding="same"),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation="relu", padding="same"),
        MaxPooling2D((2, 2)),
        
        Conv2D(128, (3, 3), activation="relu", padding="same"),
        MaxPooling2D((2, 2)),
        
        Flatten(),
        
        Dense(256, activation="relu"),
        Dropout(0.5),
        
        Dense(num_classes, activation="softmax")
    ], 
        name="TransFG_CNN")
    
    return model.summary()

In [ ]:
model_1 = build_transfg_like()

model_1.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [ ]:
# train the model
_1 = model_1.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
    callbacks= [lr_reduction , early_stop]
)

In [ ]:
plt.figure(figsize=(14, 5))

# Plot loss
plt.plot(_1._1['loss'], label='Train Loss')
plt.plot(_1._1['val_loss'], label='Val Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
plt.figure(figsize=(14, 5))

# Plot accuracy
plt.plot(_1._1['accuracy'], label='Train Accuracy')
plt.plot(_1._1['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.show()

## Model 2

In [ ]:
def build_ffvt(input_shape=input_shape, num_classes=num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), padding="same", activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), padding="same", activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), padding="same", activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), padding="same", activation="relu"),  # extra layer to simulate fusion
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation="relu"),
        Dense(num_classes, activation="softmax"),
    ], 
        name="FFVT_Sequential")
    
    return model.summary()

In [ ]:
model_2 = build_ffvt()

model_2.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [ ]:
# train the model
_2 = model_2.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
    callbacks= [lr_reduction , early_stop]
)

In [ ]:
plt.figure(figsize=(14, 5))

# Plot loss
plt.plot(_2._2['loss'], label='Train Loss')
plt.plot(_2._2['val_loss'], label='Val Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
plt.figure(figsize=(14, 5))

# Plot accuracy
plt.plot(_2._2['accuracy'], label='Train Accuracy')
plt.plot(_2._2['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.show()

## Model 3

In [ ]:
def build_smvit(input_shape=input_shape, num_classes=num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Dropout(0.2),  
        
        Conv2D(32, (3, 3), padding="same", activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), padding="same", activation="relu"),
        
        MaxPooling2D((2, 2)),
        Dropout(0.3),
        
        Conv2D(128, (3, 3), padding="same", activation="relu"),
        MaxPooling2D((2, 2)),
        
        Flatten(),
        
        Dense(128, activation="relu"),
        
        Dense(num_classes, activation="softmax"),
    ],
        name="SMViT_Sequential")
    
    return model.summary()

In [ ]:
model_3 = build_smvit()

model_3.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [ ]:
# train the model
_3 = model_3.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
    callbacks= [lr_reduction , early_stop]
)

In [ ]:
plt.figure(figsize=(14, 5))

# Plot loss
plt.plot(_3._3['loss'], label='Train Loss')
plt.plot(_3._3['val_loss'], label='Val Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
plt.figure(figsize=(14, 5))

# Plot accuracy
plt.plot(_3._3['accuracy'], label='Train Accuracy')
plt.plot(_3._3['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.show()